# 参数提取

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.2001],
        [0.1061]], grad_fn=<AddmmBackward0>)

In [3]:
net.state_dict(), net[0].state_dict()

(OrderedDict([('0.weight',
               tensor([[-0.0921, -0.2132,  0.1882, -0.3938],
                       [ 0.0174,  0.2159, -0.4064, -0.4854],
                       [ 0.3282, -0.4025, -0.1591,  0.0983],
                       [ 0.2000, -0.1587,  0.0597, -0.4587],
                       [-0.0398, -0.2723, -0.2702, -0.1506],
                       [ 0.0242,  0.2593,  0.0341,  0.3306],
                       [ 0.4925,  0.1588,  0.2328, -0.4870],
                       [ 0.0473,  0.0271, -0.3313, -0.3224]])),
              ('0.bias',
               tensor([ 0.0289,  0.3522,  0.2596, -0.2991, -0.0010, -0.0021,  0.1942, -0.3731])),
              ('2.weight',
               tensor([[ 0.3289,  0.1287, -0.0178,  0.2054,  0.2348, -0.3261, -0.0871,  0.0576]])),
              ('2.bias', tensor([0.2759]))]),
 OrderedDict([('weight',
               tensor([[-0.0921, -0.2132,  0.1882, -0.3938],
                       [ 0.0174,  0.2159, -0.4064, -0.4854],
                       [ 0.3282, -0.402

In [4]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2759], requires_grad=True)
tensor([0.2759])


In [5]:
print(net[2].weight.grad)

None


In [6]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [7]:
net.state_dict()['0.weight'].data

tensor([[-0.0921, -0.2132,  0.1882, -0.3938],
        [ 0.0174,  0.2159, -0.4064, -0.4854],
        [ 0.3282, -0.4025, -0.1591,  0.0983],
        [ 0.2000, -0.1587,  0.0597, -0.4587],
        [-0.0398, -0.2723, -0.2702, -0.1506],
        [ 0.0242,  0.2593,  0.0341,  0.3306],
        [ 0.4925,  0.1588,  0.2328, -0.4870],
        [ 0.0473,  0.0271, -0.3313, -0.3224]])

In [8]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block:{i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3512],
        [-0.3512]], grad_fn=<AddmmBackward0>)

In [9]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block:0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block:1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block:2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block:3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [10]:
rgnet[0][1][0].bias.data

tensor([ 0.4438, -0.2447,  0.4795, -0.1279, -0.3023, -0.1631, -0.2933,  0.2651])

# 初始化

In [11]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, 0, 0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]
        

(tensor([0.0015, 0.0080, 0.0016, 0.0093]), tensor(0.))

In [12]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [13]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1.0)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]


(tensor([1., 1., 1., 1.]), tensor(0.))

In [14]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.5418,  0.2490, -0.5478,  0.4069])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [15]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init ('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
Init ('weight', torch.Size([1, 8])) ('bias', torch.Size([1]))


tensor([[ 0.0000, -8.4624,  0.0000,  0.0000],
        [-0.0000,  0.0000, -0.0000,  0.0000]], grad_fn=<SliceBackward0>)

In [16]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]


tensor([42.0000, -7.4624,  1.0000,  1.0000])

# 参数绑定，共享参数

In [17]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])



tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
